## Importing

In [ ]:
import pandas as pd 
import numpy as np
import plotly.express as px
import datetime as dt
import pycountry

## Reading Data

In [3]:
data = pd.read_csv(".\Data\\aviation_accidents in countries - aviation_accidents.csv")
num_accidents = data.shape[0] # saving the count of total accidents
data.head()

Country         date       Air-craft type registration name/mark  \
0  Australia  07-Dec-2016  Boeing 737-8FE (WL)                 VH-VUP   
1    Ukraine  19-Feb-1987        Antonov An-26                    NaN   
2       Iran  19-Feb-2003     Ilyushin Il-76MD                15-2280   
3    Germany  11-May-1990      Transall C-160D                  50+39   
4     Russia  04-Jan-1943     Junkers Ju-52/3m                    NaN   

                      operator fatilites         location category  
0    Virgin Australia Airlines         0  Hobart Airpo...       A2  
1                    Soviet AF         9   near Stadnitsa       A1  
2  Iranian Revolutionary Guard       275      near Kerman       A1  
3                    German AF        10   near Rodenbach       A1  
4                    German AF         0           Ostrow       A1

## Discovering the data

In [4]:
# How many null values do we have?
nulls_count_precentage={ col_name : (data[col_name].isna().sum(), 
                                str( round( 100 * data[col_name].isna().sum() / data.shape[0] ,2))+" %") 
                                for col_name in data.columns}
nulls_count_precentage

{'Country': (0, '0.0 %'),
 'date': (68, '0.31 %'),
 'Air-craft type': (68, '0.31 %'),
 'registration name/mark': (1453, '6.68 %'),
 'operator': (70, '0.32 %'),
 'fatilites': (3067, '14.11 %'),
 'location': (419, '1.93 %'),
 'category': (71, '0.33 %')}

In [5]:
# How many unique values do we have?
data.nunique()

Country                     227
date                      14244
Air-craft type             3056
registration name/mark    19855
operator                   5697
fatilites                   359
location                  13401
category                     11
dtype: int64

## Studying the data regarding the countries

The dataset contains a column of 'Country' values with no null values, for further visualizing, we will need the code of the country. The following function does that task.

In [6]:
def get_country_code(country):
    """
    This function returns the ISO Code of a country.
    
    Argument:
    country -- A string value holds a country name
    
    Returns:
    result -- The ISO Alpha-3 code of 'country' argument.
    """
    try:
        result = pycountry.countries.search_fuzzy(country)
    except Exception:
        return np.nan
    else:
        return result[0].alpha_3
iso_map = {country: get_country_code(country) for country in data["Country"].unique()}

In [7]:
data["country_code"] = data["Country"].map(iso_map)

### Finding the number of accidents for each country

In [109]:
countries_df = data.groupby(['Country','country_code']).size().sort_values(ascending=[False])\
         .to_frame().reset_index()\
        .rename(columns= {0: 'count_accidents'})

In [119]:
# Reformating the dataframe keeping the details of only the top 15 country and combine the other countries in one row 'other'
countries_df_part=countries_df[:15]
accidents_other = countries_df['count_accidents'][15:].sum()
df2 = pd.DataFrame([['other','other', accidents_other]], columns=['Country','country_code','count_accidents'])
countries_df_part=countries_df_part.append(df2)

#### Accidents per country - visualization 1 pie chart 

For the sake of more clear visualization, only the top 15 countries were plotted with detailed information.

In [120]:
fig = px.pie(countries_df_part, 
            values='count_accidents', 
            names='Country', 
            title='15 countries where the most accidents happend')
fig.update_traces( textinfo='value+label',textfont_size=10)
fig.show()

In [116]:
#So what precentage for each country ? 
def precentage(integer):
    return integer*100/num_accidents
output_list = list(map(precentage, countries_df_part['count_accidents']))
  
print(output_list)

[19.791187563241653, 6.485143960997148, 3.845092447796891, 3.7531045901940945, 3.201177444577316, 2.7872320853647317, 2.5112685125563425, 2.341090975991169, 2.3318921902308896, 2.0927237604636186, 1.9685401526998436, 1.6189862938092172, 1.5821911507680986, 1.572992365007819, 1.4718057216447429, 40.91619906172385]


19.8% of the accidents happened in USA, 6.5% happened in Russia, 3.8% in UK and 3.7% in Canada and int the other countries 66.2% of the accident where in each of them no more than 3.2% occured

#### Accidents per country - visualization 2 Map

In [196]:
fig = px.scatter_geo(countries_df, locations="country_code",
                     color="Country", # which column to use to set the color of markers
                     hover_name="Country", # column added to hover information
                     size="count_accidents", # size of markers
                     projection="natural earth")
fig.show()

In [26]:
# Keeping the top 20 countries where the most accidents happened
countries_most_accidents = countries_df['Country'][:20]

## Studying the data regarding the operators

### Fixing operator values



Keep operator name only before the comma, some operators names are joined with city name. Ex: 'Aeroflot, Yakutsk' 

In [13]:
data["operator"]=data["operator"].str.split(',').str[0]

### Let's find the operators that had the most accidents.

In [ ]:
operators = data.groupby(['operator']).size()\
    .sort_values(ascending=[False]).to_frame()\
        .reset_index().rename(columns= {0: 'count_accidents'})

In [126]:
# reformating the dataframe, keeping the details of only th top 10 operator
operators_part=operators[:10]
accidents_other = operators['count_accidents'][10:].sum()
df2 = pd.DataFrame([['other', accidents_other]], columns=['operator','count_accidents'])
operators_part=operators_part.append(df2)

In [127]:
fig = px.pie(operators_part, 
            values='count_accidents', 
            names='operator', 
            title='10 operators which suffered the most number of accidents')
fig.update_traces( textinfo='value+label',textfont_size=10)
fig.show()

In [129]:
#So what precentage for each operator ? 
def precentage(integer):
    return round(integer*100/num_accidents,2)
output_list = list(map(precentage, operators_part['count_accidents']))
  
print(output_list)

[9.63, 5.64, 4.65, 3.88, 2.58, 1.76, 1.31, 0.8, 0.64, 0.58, 68.2]


USAAF had 9.6% of the accidents, Aerofolt had 5.6%, USAF had 4.6% and RAF had 3.9% and the other operators had 76.3% of the accidents where non of them had more than  2.6% of the accidents.

In [31]:
# Keeping the top 20 operators that had the most accidents
operators_most_accidents = operators['operator'][:20]

## Studying the data regarding the country and operators

### We're trying to figure out the count of accidents occured in each country divided by each the operators that had the accident.

In [143]:
countries_operators_df = data.groupby(['Country','country_code','operator']).size().sort_values(ascending=[False])\
    .to_frame().reset_index()\
    .rename(columns= {0: 'count_accidents'})

In [139]:
# In each country, what operators made the most accidents
fig = px.treemap(countries_operators_df, path=[px.Constant('world'), 'Country', 'operator'], values='count_accidents',
                   hover_data=['country_code'])
fig.update_traces(root_color="lightgrey")
fig.show()

### We will try to figure out the count of accidents happend by each operator divided by the country, where the accident happened.

For the sake of more clear visualization we will focus on the top 20 operators that had more the most accidents in the top 20 countries where the most accidents happened

In [ ]:
# Keeping the bair of country-operator where both the counrtry and the operator are from the top 20 of count of accidents.
countries_operators_part= countries_operators_df[countries_operators_df['Country'].isin(countries_most_accidents)]
countries_operators_part= countries_operators_part[countries_operators_df['operator'].isin(operators_most_accidents)]

In [145]:
# In each operator (of the top 20),  in which countries (of the top 20) those operators made the most accidents
fig = px.treemap(countries_operators_part, path=[px.Constant('world'), 'operator','Country'], values='count_accidents',
                   hover_data=['country_code'])
fig.update_traces(root_color="lightgrey")
fig.show()

## Studying regarding the category of the accident

### Fixing the category representation

The categories in the dataset are combined of the cause and result of the accident.
We will try to split the category into the cause and he result of the accident individually.

Accident Categories:
- A = Accident
- I = Incident
- H = Hijacking
- C = Criminal occurrence (sabotage, shoot down)
- O = other occurrence (ground fire, sabotage)
- U = type of occurrence unknown
---
- 1 = hull-loss
- 2 = repairable damage

In [22]:
# As we have seen already, there are some null values in the category colomn,
# so we will drop them first.
categories_filtered=data.dropna(subset=['category'])

In [23]:
# Defining 2 functions to add 2 colomns about the accident cause and result
def cause_accident(value):
    if value.find('A')>=0:
        return 'Accident'
    if value.find('I')>=0:
        return 'Incident'
    if value.find('H')>=0:
        return 'Hijacking'
    if value.find('C')>=0:
        return 'Criminal occurrence'
    if value.find('O')>=0:
        return 'Other occurrence'
    if value.find('U')>=0:
        return 'Unknown'

def result_accident(value):
    if value.find('1')>=0:
        return 'Hull Loss'
    if value.find('2')>=0:
        return 'Repairable Damage'

In [24]:
## Adding 2 columns representing the cause and result of the data individually
causes = categories_filtered['category'].map(cause_accident)
results = categories_filtered['category'].map(result_accident)
categories_filtered = pd.concat([categories_filtered, causes, results], axis=1, join="inner")
categories_filtered.columns = ['Country', 'date', 'Air-craft type', 'registration name/mark',
       'operator', 'fatilites', 'location', 'category', 'country_code',
       'Accident_cause', 'Accident_result']

### Studying the accidents regarding the cause and the result of them

Trying to figure out, if an air disature happened, how likely it happened because of an accident? How likely it's going to be a total loss?

In [36]:
# Finding the pair of result-cause combination
result_cause = categories_filtered.groupby(
                                ['Accident_cause','Accident_result'])  \
                                .size().sort_values(ascending=False).  \
                                to_frame().reset_index().  \
                                rename(columns={0: 'count_accidents'})

In [37]:
# the accident where the result where hull loss
hull_loss_df = result_cause[result_cause['Accident_result']=='Hull Loss']
# the accident where the result where Repairable Damage
repairable_df = result_cause[result_cause['Accident_result']=='Repairable Damage']
#
result_by_cause_df = pd.merge(hull_loss_df, repairable_df, how='outer', on = 'Accident_cause').fillna(0)
result_by_cause_df[:2]

Accident_cause Accident_result_x  count_accidents_x  \
0             Accident         Hull Loss              15813   
1  Criminal occurrence         Hull Loss               1096   

   Accident_result_y  count_accidents_y  
0  Repairable Damage             2309.0  
1  Repairable Damage               79.0

In [39]:
# Defining a list for the sake of visualization
to_draw= [result_by_cause_df["count_accidents_x"].to_list()\
         ,result_by_cause_df["count_accidents_y"].to_list()]

In [197]:
fig = px.imshow(to_draw,
                labels=dict(x="Cause of Accident", y="Result of Accident", color="Accidents"),
                x= result_by_cause_df["Accident_cause"],
                y=["Hull Loss","Repairable Damage"],
                text_auto = True
               )
fig.update_xaxes(side="top" )
fig.show()

In [42]:
# Finding the precentage of each type
def precentage(integer):
    return [round(i*100/num_accidents,2) for i in integer]
output_list = list(map(precentage, to_draw))
print(output_list)

[[72.73, 5.04, 4.32, 1.48, 0.19, 0.0], [10.62, 0.36, 0.29, 0.0, 4.59, 0.05]]


In [8]:
print("Precentage of Accidents that caused Hull Loss",round(sum (output_list[0]),2))
print("Precentage of Accidents that caused Repairable damage",round(sum(output_list[1]),2))

Precentage of Accidents that caused Hull Loss 83.76
Precentage of Accidents that caused Repairable damage 15.91


72.7% of the accidents where Accidents with hull loss, 10.6% were Accidents with Repairable damage, 5% were criminal occurence with hull loss, and 11.7% were from the rest categories, with no category more than 4.5% .

## Studying the accidents regarding the operator and cause of accidents

For the sake of more clear visualization, we will show the top 4 operators that had the most accidents.

In [153]:
category_operator= categories_filtered.groupby(
                                ['operator','Accident_result', 'Accident_cause'])\
                                .size().sort_values(ascending=False).to_frame()\
                                .reset_index().rename(columns={0: 'count_accidents'})
category_operator= category_operator[category_operator['operator'].isin(operators_most_accidents[:4])]


In [154]:
fig = px.sunburst(category_operator, path=['Accident_result','Accident_cause', 'operator'], values='count_accidents',
                  color='count_accidents')
fig.show()

Important Note : the whole circle here represent ONLY the top 4 operators and their count of accidents.

As expected, the most disasters were accidents and caused hull loss, the operators that suffered the most were USAAF, Aeroflot,and USAF.

## Studying the accidents regarding the countries and cause of accidents

For the sake of more clear visualization, we will plot the details for only the top 4 countries.

In [53]:
country_category=categories_filtered.groupby(
                      ['Country', 'Accident_result','Accident_cause']
                      ).size().sort_values(ascending=False).to_frame(
                      ).reset_index().rename(columns={0: 'count_accidents'})

country_category= country_category[country_category['Country'].isin(countries_most_accidents[:4])]


In [162]:
fig = px.bar(country_category, x="Country", y="count_accidents", color="Accident_cause", 
 pattern_shape="Accident_result", pattern_shape_sequence=["x", "."])
             
fig.show()

We can see that the majority of the disasters were Accidents with Hull loss in each of the 4 countries.

## Studying the accidents regarding the aircrafts

### Fixing Air Craft type Colomn

In [60]:
data["Air-craft type"]=data["Air-craft type"].str.split(' ').str[0]
categories_filtered["Air-craft type"]=categories_filtered["Air-craft type"].str.split(' ').str[0]

### Which Aircraft has the most accidents ?

For the sake of more clear visualisation, we will plot in details only the top 10 aircrafts, the other aircrafts will be combined in one value 'other'.

In [101]:
air_crafts = data.groupby(['Air-craft type']).size()\
    .sort_values(ascending=[False]).to_frame().reset_index()\
        .rename(columns= {0: 'count_accidents'})

In [106]:
air_craft_part=air_crafts[:10]
accidents_other = air_crafts['count_accidents'][10:].sum()
df2 = pd.DataFrame([['other', accidents_other]], columns=['Air-craft type','count_accidents'])
air_craft_part=air_craft_part.append(df2)

In [107]:
fig = px.pie(air_craft_part, 
            values='count_accidents', 
            names='Air-craft type', 
            title='10 Air crafts which suffered the most number of accidents')
fig.update_traces( textinfo='value+label',textfont_size=10)
fig.show()

In [108]:
# So what precentage for each aircraft ? 
def precentage(integer):
    return round(integer*100/num_accidents,2)
output_list = list(map(precentage, air_craft_part['count_accidents'][:12]))
print(output_list)
print(sum(output_list))

[23.77, 7.58, 7.22, 5.51, 4.0, 3.48, 2.79, 2.62, 2.34, 2.1, 38.28]
99.69


Douglas had 23.8% of the accidents, Antonov had 7.6%, Boeing had 7.2%, and all the other air crafts caused 61% of the accidents with each of them causing less than 6%

In [187]:
# Keeping the top 4 aircrafts with most accidents
air_craft_top = air_craft_part[:4]

## Studying air craft and accident categories
Trying to figure out, if an air crises happened with a certain type of air craft type, how likely is it going to be a hull loss?

In [188]:
air_craft_category=categories_filtered.groupby(
                      ['Air-craft type', 'Accident_result','Accident_cause']
                      ).size().sort_values(ascending=False).to_frame(
                      ).reset_index().rename(columns={0: 'count_accidents'})

In [194]:
air_craft_category_filtered= air_craft_category[air_craft_category['Air-craft type'].isin(air_craft_top['Air-craft type'])]

In [193]:
air_craft_top

Air-craft type  count_accidents
0        Douglas             5169
1        Antonov             1649
2         Boeing             1570
3       Lockheed             1197

In [195]:
fig = px.bar(air_craft_category_filtered, x="Air-craft type",\
     y="count_accidents", color="Accident_cause", 
 pattern_shape="Accident_result", pattern_shape_sequence=[ "x","."])
fig.show()

From the previous we can see that for each air-craft type, when the flight faced a crises it was an accident for the majority of the time and it caused a hull loss, as it was shown from studying the accidents types generally without focusing on the air-craft type.

## Studying the data regarding the date

### Editing the date value
The date is given in day-month-year format, some of the values are missing, they are either Null values or explicitly written 'Date unk'. Besides, some of the values contians question marks where a part of the date is missing. 

We will create a new colomn contains the year data, still we'll have a lot of years. Thus, we will add another colomn that contains the decade of when the accident happened.

In [156]:
#### Dropping rows with unknow dates
dates_filtered = data.drop(data[(data.date == 'date unk.') ].index)
#### Dropping rows with null dates
dates_filtered=dates_filtered.dropna(subset=['date'])

In [157]:
#### Adding a year & decade colomns
dates_filtered['year'] = dates_filtered['date'].str[-4:].str.lower()
dates_filtered['decade'] = dates_filtered['year'].astype(int).round(-1)

### Finding the number of accidents for each country in each decade

In [176]:
decade_country= dates_filtered.groupby(['Country','country_code',\
    'decade']).size().to_frame()\
    .reset_index().rename(columns={0: 'accidents'})

In [199]:
fig = px.line(decade_country, x="decade", y="accidents", color="Country", line_group="Country", hover_name="Country",
        line_shape="spline", render_mode="svg")
fig.show()

In [200]:
# Anthore type of representation

df = px.data.medals_long()
fig = px.bar(decade_country, x="decade", y="accidents", color="Country")           
fig.show()

In the sake of more clear visualization, let's draw only the top 14 countries in details, and combine the rest in 'other' colomn

In [178]:
# Countries to be shown
decade_show_country=decade_country[decade_country['Country'].isin(countries_most_accidents[:14])]

In [203]:
#countries to be combined
decade_other_country =decade_country[~decade_country['Country'].isin(countries_most_accidents[:14])]
decade_other_country = decade_other_country.groupby('decade').size().to_frame()\
    .reset_index().rename(columns={0: 'accidents'})
decade_other_country.insert(0, 'Country', 'Other')
decade_other_country.insert(1, 'country_code', 'Other')
decade_other_country[:2]

Country country_code  decade  accidents
0   Other        Other    1920          6
1   Other        Other    1930         20

In [202]:
# the final dataframe to be plotted.
decade_country_finale = pd.concat([decade_show_country,decade_other_country])

In [183]:
fig = px.bar(decade_country_finale, x="decade", y="accidents", color="Country")
fig.show()

We can see that the number of accidents in around 1940 were significantly higher.